In [1]:
!uname -m
!arch
!which clang++
!clang++ --version

arm64
arm64
/usr/bin/clang++
Apple clang version 16.0.0 (clang-1600.0.26.6)
Target: arm64-apple-darwin24.2.0
Thread model: posix
InstalledDir: /Applications/Xcode_16.2.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin


In [3]:
%load_ext autoreload
%autoreload 2

%aimport pytensor

In [4]:
import pymc as pm
import pymc_bart as pmb
import numpy as np

In [9]:
import numpy as np
from pymc_bart.utils import MutableDataWithNA
import pymc as pm
import pymc_bart as pmb

# Create data with missing values
X = np.array([[1.0, 2.0, np.nan], [4.0, np.nan, 6.0], [7.0, 8.0, 9.0]])
Y = np.array([1.0, 2.0, 3.0])

# Create mutable data container
data_X = MutableDataWithNA("X", X)

# Use directly with BART (bypassing pm.Data for missing values)
with pm.Model() as model:
    mu = pmb.BART("mu", data_X.get_value(), Y, m=10, missingness_handling="aware")
    sigma = pm.HalfNormal("sigma", 1)
    y = pm.Normal("y", mu, sigma, observed=Y, shape=mu.shape)
    
    # Sample
    idata = pm.sample(tune=100, draws=100, chains=1)
    
    # Update data for predictions
    new_X = np.array([[10.0, 20.0, np.nan], [40.0, 50.0, 60.0], [70.0, 80.0, 90.0]])
    data_X.set_value(new_X)
    
    # Sample posterior predictive
    ppc = pm.sample_posterior_predictive(idata)

Sequential sampling (1 chains in 1 job)
CompoundStep
>PGBART: [mu]
>NUTS: [sigma]


Output()

Sampling 1 chain for 100 tune and 100 draw iterations (100 + 100 draws total) took 2 seconds.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
Only one chain was sampled, this makes it impossible to run some convergence checks
Sampling: [y]


Output()

In [5]:
# Create data with missing values
X = np.random.normal(0, 1, size=(100, 3))
X[10:20, 0] = np.nan  # Add missing values
Y = np.random.normal(0, 1, size=100)

In [7]:

# Create data with missing values
X = np.random.normal(0, 1, size=(100, 3))
X[10:20, 0] = np.nan  # Add missing values
Y = np.random.normal(0, 1, size=100)

# Use missingness-aware BART (bartMachine-like)
with pm.Model() as model:
    mu = pmb.BART(
        "mu", 
        X, 
        Y, 
        m=10, 
        missingness_handling="aware",  # New parameter!
        auto_detect_categorical=True   # New parameter!
    )
    sigma = pm.HalfNormal("sigma", 1)
    pm.Normal("y", mu, sigma, observed=Y)
    idata = pm.sample(tune=100, draws=100, chains=1)

Sequential sampling (1 chains in 1 job)
CompoundStep
>PGBART: [mu]
>NUTS: [sigma]


Output()

Sampling 1 chain for 100 tune and 100 draw iterations (100 + 100 draws total) took 3 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks
